In [7]:
import tensorflow as tf
# 移除不使用的 import
# from tensorflow.keras.applications.resnet_v2 import preprocess_input
from roboflow import Roboflow
import cv2
import numpy as np
import os
import time

# --- 步驟一：參數設定 ---
# 將所有可調整的參數集中管理
class Config:
    ROBOFLOW_API_KEY = "UCLBeCClmaD7BW6BWuLG" # 建議從環境變數讀取，避免金鑰外洩
    PROJECT_ID = "potato-detection-3et6q"
    MODEL_VERSION = 11
    CLASSIFIER_MODEL_PATH = '../best_potato_model.keras'
    CLASSIFIER_IMG_SIZE = (224, 224)
    # 透過批次處理優化後，可以更頻繁地進行偵測
    PROCESS_EVERY_N_FRAMES = 3
    # 分類模型的類別名稱 (假設 0 是 'Not Sprouted', 1 是 'Sprouted')
    CLASS_NAMES = ["Not Sprouted", "Sprouted"]
    # 顏色管理
    COLORS = {
        "Sprouted": (0, 0, 255),       # 紅色
        "Not Sprouted": (0, 255, 0) # 綠色
    }

# --- 步驟二：模型載入與攝影機選擇 (邏輯不變，僅微調) ---

def load_models(config):
    """載入所有模型"""
    try:
        print("🔌 正在載入模型...")
        rf = Roboflow(api_key=config.ROBOFLOW_API_KEY)
        project = rf.workspace().project(config.PROJECT_ID)
        detection_model = project.version(config.MODEL_VERSION).model
        classifier_model = tf.keras.models.load_model(config.CLASSIFIER_MODEL_PATH)
        print("✅ 所有模型載入成功！")
        return detection_model, classifier_model
    except Exception as e:
        print(f"❌ 模型載入失敗: {e}")
        return None, None

def find_and_select_camera(max_cameras_to_check=5):
    """偵測並讓使用者選擇攝影機 (與原版邏輯相同)"""
    # ... 此處省略您原本的 find_available_cameras 和 select_camera 函式程式碼 ...
    # 為了簡潔，直接整合在一起
    available_cameras = []
    print("🔍 正在尋找可用的攝影機...")
    for i in range(max_cameras_to_check):
        cap = cv2.VideoCapture(i, cv2.CAP_DSHOW)
        if cap.isOpened():
            print(f"  ✅ 找到攝影機，索引為: {i}")
            available_cameras.append(i)
            cap.release()
        else:
            break
    if not available_cameras:
        print("❌ 找不到任何攝影機。")
        return None
    if len(available_cameras) == 1:
        print(f"✅ 自動選擇唯一的攝影機，索引為: {available_cameras[0]}")
        return available_cameras[0]
    
    # 預覽邏輯可保留或簡化
    while True:
        try:
            choice = input(f"\n👉 請從可用的索引 {available_cameras} 中輸入您想使用的攝影機編號: ")
            selected_index = int(choice)
            if selected_index in available_cameras:
                print(f"✅ 您已選擇使用攝影機，索引為: {selected_index}")
                return selected_index
            else:
                print(f"❌ 無效的選擇。")
        except ValueError:
            print("❌ 輸入無效，請輸入一個數字。")

# --- 步驟三：優化後的核心處理函式 ---

def process_frame_with_batching(frame, detection_model, classifier_model, config):
    """
    對單一畫面進行偵測與批次分類，回傳繪圖所需的資訊。
    """
    # 1. 準備偵測
    original_h, original_w, _ = frame.shape
    # Roboflow 模型通常在 640x640 訓練，可直接傳入原始幀，它會自動縮放
    # 移除不必要的磁碟寫入
    predictions = detection_model.predict(frame, confidence=40, overlap=30).json()['predictions']
    
    crops_to_classify = []
    original_boxes = []

    # 2. 裁切所有偵測到的物件
    for pred in predictions:
        center_x, center_y = int(pred['x']), int(pred['y'])
        width, height = int(pred['width']), int(pred['height'])
        x1 = max(0, int(center_x - width / 2))
        y1 = max(0, int(center_y - height / 2))
        x2 = min(original_w, int(center_x + width / 2))
        y2 = min(original_h, int(center_y + height / 2))
        
        cropped_potato = frame[y1:y2, x1:x2]
        
        if cropped_potato.size > 0:
            resized_crop = cv2.resize(cropped_potato, config.CLASSIFIER_IMG_SIZE)
            rgb_crop = cv2.cvtColor(resized_crop, cv2.COLOR_BGR2RGB)
            crops_to_classify.append(rgb_crop)
            original_boxes.append((x1, y1, x2, y2))

    # 3. [效能優化] 進行批次分類
    render_info = []
    if crops_to_classify:
        # 將圖片列表轉換成一個 (N, H, W, C) 的批次
        batch_images = np.array(crops_to_classify)
        
        # 一次性預測所有圖片
        batch_predictions = classifier_model.predict(batch_images, verbose=0)
        
        for i, score in enumerate(batch_predictions):
            prediction_score = score[0]
            box = original_boxes[i]
            
            # 根據分數決定標籤和顏色
            if prediction_score > 0.5:
                class_name = config.CLASS_NAMES[1] # Sprouted
                label = f"{class_name}: {prediction_score:.2f}"
            else:
                class_name = config.CLASS_NAMES[0] # Not Sprouted
                label = f"{class_name}: {1-prediction_score:.2f}"
            
            color = config.COLORS.get(class_name, (255, 255, 255)) # 預設白色
            render_info.append((box, label, color))
            
    return render_info

# --- 步驟四：主迴圈 ---

def main():
    config = Config()
    detection_model, classifier_model = load_models(config)
    
    if not (detection_model and classifier_model):
        return

    selected_cam_index = find_and_select_camera()
    if selected_cam_index is None:
        return

    cap = cv2.VideoCapture(selected_cam_index, cv2.CAP_DSHOW)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    if not cap.isOpened():
        print(f"❌ 錯誤：無法打開攝影機 {selected_cam_index}。")
        return

    print("\n📹 攝影機已啟動！按下 'q' 鍵退出。")
    
    frame_counter = 0
    latest_render_info = []

    while True:
        ret, frame = cap.read()
        if not ret: break

        if frame_counter % config.PROCESS_EVERY_N_FRAMES == 0:
            latest_render_info = process_frame_with_batching(frame, detection_model, classifier_model, config)
        
        # 使用最新的繪圖資訊來更新畫面
        for box, label, color in latest_render_info:
            (x1, y1, x2, y2) = box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        cv2.imshow('Real-time Potato Sprout Detection', frame)
        
        frame_counter += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    # 確保所有視窗都已關閉
    for i in range(5):
        cv2.waitKey(1)
    print("👋 程式已結束。")

if __name__ == "__main__":
    main()

🔌 正在載入模型...
loading Roboflow workspace...
loading Roboflow project...
✅ 所有模型載入成功！
🔍 正在尋找可用的攝影機...
  ✅ 找到攝影機，索引為: 0
  ✅ 找到攝影機，索引為: 1
  ✅ 找到攝影機，索引為: 2



👉 請從可用的索引 [0, 1, 2] 中輸入您想使用的攝影機編號:  0


✅ 您已選擇使用攝影機，索引為: 0

📹 攝影機已啟動！按下 'q' 鍵退出。
👋 程式已結束。
